# Prototipo simulador parte 2: Intercambio gaseoso

Eventualmente, las clases creadas al final del *notebook* 1, se grabarán en un módulo con extensión `.py`, de forma que puedan ser importadas al inicio de cada notebook, así:

```python
from scipy.integrate import solve_ivp
from paciente import Paciente
from ventilador import Ventilador
```  
En ese punto, su implementación sería más o menos así:

```python
paciente_sano = Paciente()
ventilador_config = Ventilador(modo='PCV')
sim = Simulador(paciente_sano, ventilador_config)
resultados_ciclo = sim.correr_ciclo()
```

Sin embargo, por ahora, se copian y pegan en este cuaderno, a continuación:

In [1]:
# Librerías
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Estilo
plt.style.use('seaborn-v0_8-whitegrid')

In [2]:
class Paciente:
    """Clase que define un paciente con parámetros normales;
    para agregar pacientes con patologías, crear una subclase con modificaciones
    en los parámetros específicos que haga falta"""
    def __init__(self, R1=5, C1=0.05, R2=10, C2=0.05):
        self.R1 = R1
        self.C1 = C1
        self.R2 = R2
        self.C2 = C2
        self.E1 = 1 / self.C1
        self.E2 = 1 / self.C2

In [ ]:
class Ventilador:
    """Parámetros y perfiles de ventilación mecánica."""
    def __init__(self,
                 modo: str = 'PCV',
                 PEEP: float = 5.0,
                 P_driving: float = 15.0,
                 fr: float = 20.0,
                 Ti: float = 1.0,
                 Vt: Optional[float] = None):
        self.modo = modo
        self.PEEP = PEEP
        self.P_driving = P_driving
        self.fr = fr
        self.Ti = Ti
        self.T_total = 60.0 / fr
        self.Vt = Vt
        if modo == 'VCV':
            assert Vt is not None, "Se requiere Vt para modo VCV"
            self.flow_insp = Vt / Ti
        else:
            self.flow_insp = None

    def presion(self, t: Union[float, np.ndarray]) -> np.ndarray:
        """Perfil de presión en la vía aérea según el modo y el tiempo t."""
        t_arr = np.asarray(t)
        en_insp = (t_arr % self.T_total) < self.Ti
        if self.modo == 'PCV':
            P_control = self.PEEP + self.P_driving
            return np.where(en_insp, P_control, self.PEEP)
        elif self.modo == 'VCV':
            return np.full_like(t_arr, self.PEEP)
        else:
            raise ValueError(f"Modo desconocido: {modo}")

    def flujo(self, t: Union[float, np.ndarray]) -> np.ndarray:
        """Perfil de flujo inspirado en VCV, 0 fuera de inspiración."""
        if self.modo != 'VCV':
            return np.zeros_like(np.asarray(t))
        t_arr = np.asarray(t)
        return np.where((t_arr % self.T_total) < self.Ti, self.flow_insp, 0.0)

In [ ]:
class Simulador:
    """Orquesta la simulación paciente-ventilador."""
    def __init__(self,
                 paciente: Paciente,
                 ventilador: Ventilador):
        self.paciente = paciente
        self.ventilador = ventilador

    def _modelo_edo(self, t, y):
        t_arr = np.asarray(t)
        V1, V2 = y

        # Detectar inspiración vs espiración
        en_insp = (t_arr % self.ventilador.T_total) < self.ventilador.Ti

        if self.ventilador.modo == 'VCV':
            # Flujo: flujo constante en inspiración, cero en espiración
            flow_total = np.where(en_insp,
                                  self.ventilador.flow_insp,
                                  0.0)

            # Presión de vía aérea calculada por la ecuación de los dos compartimentos
            P_aw_insp = (
                flow_total
                + (self.paciente.E1 * V1 / self.paciente.R1)
                + (self.paciente.E2 * V2 / self.paciente.R2)
            ) / ((1.0 / self.paciente.R1) + (1.0 / self.paciente.R2))

            P_aw = np.where(en_insp,
                            P_aw_insp,
                            self.ventilador.PEEP)

        else:  # Modo PCV
            P_aw = self.ventilador.presion(t_arr)

        # Se calculan las derivadas dV/dt
        dV1_dt = (P_aw - self.paciente.E1 * V1) / self.paciente.R1
        dV2_dt = (P_aw - self.paciente.E2 * V2) / self.paciente.R2

        return [dV1_dt, dV2_dt]

    def simular(self,
                num_ciclos: int=15,
                pasos_por_ciclo: int = 100):
        """Ejecuta múltiples ciclos y devuelve t, V1 y V2 concatenados."""
        t_data, V1_data, V2_data = [], [], []
        V0 = [0.0, 0.0]
        for i in range(num_ciclos):
            t0 = i * self.ventilador.T_total
            t1 = (i + 1) * self.ventilador.T_total
            if i < num_ciclos-1: #corrección para evitar discontinuidad en t=3.0
                t_eval = np.linspace(t0, t1, pasos_por_ciclo, endpoint=False)
            else:
                t_eval = np.linspace(t0, t1, pasos_por_ciclo, endpoint=True)
            sol = solve_ivp(self._modelo_edo,
                            [t0, t1],
                            V0,
                            t_eval=t_eval)
            t_data.append(sol.t)
            V1_data.append(sol.y[0])
            V2_data.append(sol.y[1])
            V0 = sol.y[:, -1]
        t = np.concatenate(t_data)
        V1 = np.concatenate(V1_data)
        V2 = np.concatenate(V2_data)
        return t, V1, V2

    def procesar_resultados(self,
                            t: np.ndarray,
                            V1: np.ndarray,
                            V2: np.ndarray) -> dict:
        """Calcula flujo, volumen total y presión resultante."""
        flujo1 = np.gradient(V1, t)
        flujo2 = np.gradient(V2, t)
        flujo_total = flujo1 + flujo2
        Vt = V1 + V2
        if self.ventilador.modo == 'PCV':
            P_aw = self.ventilador.presion(t)
        else:
            P_aw = ((flujo_total
                     + (self.paciente.E1 * V1 / self.paciente.R1)
                     + (self.paciente.E2 * V2 / self.paciente.R2))
                    / ((1 / self.paciente.R1) + (1 / self.paciente.R2)))
        return {
            't': t,
            'V1': V1,
            'V2': V2,
            'Vt': Vt,
            'flow1': flujo1,
            'flow2': flujo2,
            'flow': flujo_total,
            'P_aw': P_aw
        }

    def graficar_resultados(self,
                            resultados: dict,
                            titulo: str = 'Simulación Pulmonar'):
        """Grafica presión, flujo total y volumen total a partir del diccionario de resultados."""
        t = resultados['t']
        P_aw = resultados['P_aw']
        flujo = resultados['flow']
        Vt = resultados['Vt']

        fig, axs = plt.subplots(3, 1, figsize=(15, 10), sharex=True)
        fig.suptitle(titulo, fontsize=16)

        # Presión
        axs[0].plot(t, P_aw, color='red', label='Presión (P_aw)')
        axs[0].set_ylabel('Presión (cmH2O)')
        axs[0].legend()

        # Flujo
        axs[1].plot(t, flujo, color='blue', label='Flujo Total')
        axs[1].set_ylabel('Flujo (L/s)')
        axs[1].axhline(0, color='grey', linewidth=0.8)
        axs[1].legend()

        # Volumen
        axs[2].plot(t, Vt, color='green', label='Volumen (L)')
        axs[2].set_ylabel('Volumen (L)')
        axs[2].set_xlabel('Tiempo (s)')
        axs[2].legend()

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.show()